In [201]:
import sys

# ***** change path below as necessary *****
sys.path.append("/Users/herman/Documents/CDIPS-DSW-2015/DataScience-2015_self/")

import acronym_extract
import dewiki, re

In [2]:
import pyspark
from pyspark.sql import SQLContext
sqlCtx = SQLContext(sc)

In [3]:
from pyspark.sql import Row

In [4]:
wikiData = sqlCtx.read.parquet("./wiki_parquet")

In [5]:
wikiData.printSchema()

root
 |-- id: integer (nullable = true)
 |-- title: binary (nullable = true)
 |-- modified: long (nullable = true)
 |-- text: binary (nullable = true)
 |-- username: binary (nullable = true)



In [6]:
wikiData.count()

39365

In [7]:
wikiData.registerTempTable("wikiData")

In [8]:
resultUCB = sqlCtx.sql("SELECT * FROM wikiData WHERE title LIKE '%University of California, Berkeley%'")

In [23]:
UCB_Text = resultUCB.map(lambda p: str(p.text)).collect()

In [10]:
UCB_Title = resultUCB.map(lambda p: str(p.title)).collect()

In [171]:
UCB = UCB_Text[10]

In [203]:
# GETTING INDICES FROM REGEX SEARCH (re.finditer())

string = '{{asdfasdf{{asdf}}asdfasdf}}asdfasdf{{s}}asdf{{asdf{{asdf{{asdf}}}}}}'
braces = re.findall('{{|}}', string)
indices = [(m.start(0), m.end(0)) for m in re.finditer('{{|}}', string)]

print(braces)
print(indices)


['{{', '{{', '}}', '}}', '{{', '}}', '{{', '{{', '{{', '}}', '}}', '}}']
[(0, 2), (10, 12), (16, 18), (26, 28), (36, 38), (39, 41), (45, 47), (51, 53), (57, 59), (63, 65), (65, 67), (67, 69)]


In [197]:
def cleanWikiMarkup(text):
    RE_BRACKETS = re.compile(r'{{(?:.(?!{{|}}).)*}}')#  {{}} and everything inside as long as there are no nested {{}}
    RE = re.compile(r'''\[{2}(File|Category):.+?\]{2}|        #  [[File: ]]
                                    [\s\w#()]+?\||            #  
                                    (\[{2}|\]{2})|            #  [[ or ]]
                                    \'{2,5}|                  #  two to five occurences of '
                                    (<s>|<!--).+?(</s>|-->)|  #  comments
                                    ={1,6}|                   #  one to six occurences of =
                                    \<ref.+?(\/>|\<\/ref\>)|  #  <ref /> and <ref></ref> tags
                                    \<.+?\>                   #  any tags
                                    ''',   
                                    re.X)

#    text = re.sub('\{\{[^\}]+?(?=\{\{)|(?<=\}\})[^\{]+?\}\}', ' ', text)
#    double_braces = re.findall(r'{{[^}]+?{{.+?}}[^}]+?}}', text)
#    print(double_braces)
#    for db in double_braces:
#        to_delete = re.match(r'^{{.+?({{.+?}}).+?}}$', text).group()
#        db_new = re.sub(to_delete, '', db)
#        text = re.sub(db, db_new, text)
        
#    text = re.sub('{{.+?}}', ' ', text)
#    else:
#        while re.search('[\{\}]', text) != None:    # recursively delete nested brackets from insidemost out
#            text = re.sub(RE_BRACKETS, ' ', text)

    text = re.sub(RE, ' ', text)
    text = re.sub('\s+', ' ', text)
    text = re.sub('\( ', '(', text)
    text = re.sub(' \)', ')', text)
    text = re.sub('\|', '| ', text) 
    return text


In [198]:
UCB_clean = cleanWikiMarkup(UCB)

In [199]:
UCB_clean

' Image:UCB-Reserved-For-NL. Nobel laureates on Berkeley\'s faculty can take advantage of special exclusive parking spaces on the Berkeley campus. This is an alphabetical table of Nobel laureates associated with the University of California, Berkeley , including current and former faculty members and researchers as well as graduates. , 72 Nobel laureates have been associated with Berkeley.Berkeley currently has nine Nobel laureates on its faculty, out of the twenty-five full-time faculty members who have won the Prize. Sortable table {|  class "wikitable sortable" style "background: white; padding: 0.5em;"!Name!Year!Prize! -|  (1940\xe2\x80\x93 |  |  |  Professor of Economics (1966\xe2\x80\x931978, 1980\xe2\x80\x93 - (1911\xe2\x80\x93 |  |  |  Professor of Physics (1936\xe2\x80\x93 - (1929\xe2\x80\x93 |  |  |  Researcher (1963), Visiting Professor of Molecular Biology (1970\xe2\x80\x93 -|  (1948\xe2\x80\x93 |  |  |  Professor of Molecular Biology (1978\xe2\x80\x93 -|  (1905\xe2\x80\x93

In [12]:
acronym_extract.extract_acronym(UCB_Text[8])

[]

In [15]:
acronym_extract.extract_acronym(temptext)

[['ARWU', 'Academic Ranking of World Universities', ''],
 ['ARRA', 'American Reinvestment and Recovery Act', 'considered'],
 ['ASUC', 'Associated Students of the University of California', 'into'],
 ['SUPERB',
  'Student Union Program, Entertainment, and Recreation Board',
  'to'],
 ['RHA', 'Residence Hall Assembly', 'are']]